In [2]:
#import libairies
import pandas as pd
import numpy as np
import seaborn as sns
import random
import warnings
import matplotlib.pyplot as plt
from datetime import timedelta, date
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from prettytable import PrettyTable
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.svm import OneClassSVM
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
plt.rcParams.update({'figure.figsize': (10,10), 'figure.dpi':600,})
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1

In [3]:
#read the data using the following function. 
data = pd.read_csv('divorces_2000-2015_translated.csv') #just give the proper path 

In [4]:
#info about the data.
data.info()

#find no of null on each columns
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4872 entries, 0 to 4871
Data columns (total 41 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Divorce_date                          4872 non-null   object 
 1   Type_of_divorce                       4872 non-null   object 
 2   Nationality_partner_man               4871 non-null   object 
 3   DOB_partner_man                       4496 non-null   object 
 4   Place_of_birth_partner_man            4746 non-null   object 
 5   Birth_municipality_of_partner_man     4743 non-null   object 
 6   Birth_federal_partner_man             4744 non-null   object 
 7   Birth_country_partner_man             4745 non-null   object 
 8   Age_partner_man                       4766 non-null   float64
 9   Residence_municipality_partner_man    4551 non-null   object 
 10  Residence_federal_partner_man         4552 non-null   object 
 11  Residence_country

Divorce_date                               0
Type_of_divorce                            0
Nationality_partner_man                    1
DOB_partner_man                          376
Place_of_birth_partner_man               126
Birth_municipality_of_partner_man        129
Birth_federal_partner_man                128
Birth_country_partner_man                127
Age_partner_man                          106
Residence_municipality_partner_man       321
Residence_federal_partner_man            320
Residence_country_partner_man            321
Monthly_income_partner_man_peso         1408
Occupation_partner_man                   523
Place_of_residence_partner_man           318
Nationality_partner_woman                  3
DOB_partner_woman                        447
DOB_registration_date_partner_woman     2652
Place_of_birth_partner_woman             140
Birth_municipality_of_partner_woman      139
Birth_federal_partner_woman              140
Birth_country_partner_woman              139
Age_partne

In [5]:
#preprocessing of the dataset to fill the null value.
data= data.dropna(subset = ['Nationality_partner_man','Nationality_partner_woman','Age_partner_man','Age_partner_woman',
                           'Marriage_duration','Place_of_birth_partner_man','Place_of_birth_partner_woman'])
data= data.dropna(subset = ['Place_of_birth_partner_man','Place_of_birth_partner_woman','Place_of_residence_partner_man',
                           'Place_of_residence_partner_woman'])
data = data.astype({'Divorce_date':'datetime64[ns]','Date_of_marriage':'datetime64[ns]',
                   'DOB_partner_man':'datetime64[ns]', 'DOB_partner_woman':'datetime64[ns]'})

data['Num_Children'] = data['Num_Children'].fillna(0).astype(int)
data['Custody'] = data['Custody'].fillna("NA")
data['Monthly_income_partner_man_peso']= data['Monthly_income_partner_man_peso'].fillna(data.Monthly_income_partner_man_peso.min())
data['Monthly_income_partner_woman_peso']= data['Monthly_income_partner_woman_peso'].fillna(data.Monthly_income_partner_woman_peso.min())
data['Occupation_partner_man'] = data['Occupation_partner_man'].fillna(data.Occupation_partner_man.mode()[0])
data['Occupation_partner_woman'] = data['Occupation_partner_woman'].fillna(data.Occupation_partner_woman.mode()[0])
data['Employment_man'] = data["Employment_status_partner_man"].replace(np.nan, data.Employment_status_partner_man.mode()[0])
data['Employment_woman'] = data["Employment_status_partner_woman"].replace(np.nan, data.Employment_status_partner_woman.mode()[0])
data['Education_man'] = data["Level_of_education_partner_man"].replace(np.nan, data.Level_of_education_partner_man.mode()[0])
data['Education_woman'] = data["Employment_status_partner_woman"].replace(np.nan, data.Level_of_education_partner_woman.mode()[0])
data['maritial_status'] = 0

In [6]:
#create another dataframe for important features.
Processed_data = data[['Nationality_partner_man','Nationality_partner_woman','Age_partner_man','Age_partner_woman',
                           'Marriage_duration','Place_of_birth_partner_man','Place_of_birth_partner_woman',
                           'Place_of_residence_partner_man','Place_of_residence_partner_woman',
                           'Monthly_income_partner_man_peso','Monthly_income_partner_woman_peso',
                           'Occupation_partner_man','Occupation_partner_woman',
                           'Employment_man','Employment_woman','Education_man','Education_woman',
                           'Num_Children','Custody']]
Processed_data

,Nationality_partner_man,Nationality_partner_woman,Age_partner_man,Age_partner_woman,Marriage_duration,Place_of_birth_partner_man,Place_of_birth_partner_woman,Place_of_residence_partner_man,Place_of_residence_partner_woman,Monthly_income_partner_man_peso,Monthly_income_partner_woman_peso,Occupation_partner_man,Occupation_partner_woman,Employment_man,Employment_woman,Education_man,Education_woman,Num_Children,Custody
0,MEXICANA,MEXICANA,30.0,22.0,5.0,XALAPA - ENRIQUEZ,PUEBLA,XALAPA-ENRIQUEZ,XALAPA-ENRIQUEZ,2000.0,1800.0,PINTOR,EMPLEADA,OBRERO,EMPLEADO,SECUNDARIA,EMPLEADO,1,NA
4,MEXICANA,MEXICANA,30.0,27.0,7.0,XALAPA - ENRIQUEZ,XALAPA-ENRIQUEZ,COATEPEC,COATEPEC,18000.0,3.5,MEDICO,AMA DE CASA,EMPLEADO,NO TRABAJA,PROFESIONAL,NO TRABAJA,2,MADRE
5,MEXICANA,MEXICANA,32.0,29.0,11.0,COATEPEC,RANCHO NUEVO,XALAPA-ENRIQUEZ,XALAPA-ENRIQUEZ,3.0,3.5,EMPLEADO,EMPLEADA,EMPLEADO,NO TRABAJA,SECUNDARIA,NO TRABAJA,2,MADRE
6,MEXICANA,MEXICANA,36.0,35.0,15.0,XALAPA - ENRIQUEZ,XALAPA-ENRIQUEZ,XALAPA-ENRIQUEZ,PIEDRA DE AGUA,3.0,3.5,EMPLEADO,LABORES DOMESTICAS,EMPLEADO,NO TRABAJA,PROFESIONAL,NO TRABAJA,2,MADRE
7,MEXICANA,MEXICANA,31.0,32.0,6.0,DISTRITO FEDERAL,DISTRITO FEDERAL,XALAPA-ENRIQUEZ,XALAPA-ENRIQUEZ,3.0,3.5,LICENCIADO,LICENCIADA,EMPLEADO,EMPLEADO,PROFESIONAL,EMPLEADO,1,MADRE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4867,MEXICANA,MEXICANA,35.0,34.0,3.0,MORELIA,XALAPA-ENRIQUEZ,MORELIA,XALAPA-ENRIQUEZ,12500.0,6000.0,AUDITOR,EMPLEADA,EMPLEADO,EMPLEADO,PROFESIONAL,EMPLEADO,0,NA
4868,MEXICANA,MEXICANA,30.0,32.0,1.0,VERACRUZ,TLAPACOYAN,XALAPA-ENRIQUEZ,XALAPA-ENRIQUEZ,3.0,3.5,EMPLEADO,EMPLEADA,EMPLEADO,EMPLEADO,PROFESIONAL,EMPLEADO,0,NA
4869,MEXICANA,MEXICANA,59.0,56.0,35.0,XALAPA - ENRIQUEZ,XALAPA-ENRIQUEZ,EMILIANO ZAPATA,XALAPA-ENRIQUEZ,20000.0,29000.0,MEDICO,JUBILADA,EMPLEADO,EMPLEADO,PROFESIONAL,EMPLEADO,2,NA
4870,MEXICANA,MEXICANA,54.0,48.0,33.0,TLAPACOYAN,TLAPACOYAN,XALAPA-ENRIQUEZ,XALAPA-ENRIQUEZ,3.0,3.5,EMPLEADO,AMA DE CASA,EMPLEADO,NO TRABAJA,PREPARATORIA,NO TRABAJA,2,NA


In [7]:
#finding categorical and numerical features of the data.
cols = Processed_data.columns
num_cols = Processed_data._get_numeric_data().columns
rem_cols = list(set(cols) - set(num_cols))
num_cols
rem_cols

Index(['Age_partner_man', 'Age_partner_woman', 'Marriage_duration',
       'Monthly_income_partner_man_peso', 'Monthly_income_partner_woman_peso',
       'Num_Children'],
      dtype='object')

['Education_man',
 'Occupation_partner_woman',
 'Place_of_residence_partner_woman',
 'Education_woman',
 'Occupation_partner_man',
 'Place_of_residence_partner_man',
 'Custody',
 'Place_of_birth_partner_man',
 'Nationality_partner_man',
 'Place_of_birth_partner_woman',
 'Employment_man',
 'Employment_woman',
 'Nationality_partner_woman']

In [8]:
#Creating the box plot of the fetaures.
colors = ["g","brown","r","c","blue","y"]
all_data = [Processed_data.Age_partner_man,Processed_data.Age_partner_woman]
all_data1 = [Processed_data.Marriage_duration,]
all_data2 = [Processed_data.Monthly_income_partner_man_peso,Processed_data.Monthly_income_partner_woman_peso,]
all_data3 = [Processed_data.Num_Children]

fig, ax = plt.subplots(2,2,figsize=(15, 15))
bplot1 = ax[0,0].boxplot(all_data,
                         vert=True,  # vertical box alignment
                         patch_artist=True,  # fill with color
                       labels =['Man_age','Woman_age'], showfliers=True)  # will be used to label x-ticks
bplot2= ax[0,1].boxplot(all_data1,
                         vert=True,  # vertical box alignment
                         patch_artist=True,  # fill with color
                       labels =['Marraige Duration'], showfliers=True)  # will be used to label x-ticks
bplot3 = ax[1,0].boxplot(all_data2,
                         vert=True,  # vertical box alignment
                         patch_artist=True,  # fill with color
                       labels =['Monthly income man','Monthly income woman'], showfliers=False)  # will be used to label x-ticks
bplot4= ax[1,1].boxplot(all_data3,
                         vert=True,  # vertical box alignment
                         patch_artist=True,  # fill with color
                       labels =['Num_Children'], showfliers=True)  # will be used to label x-ticks

for patch, color in zip(bplot1['boxes'], colors):
    patch.set_facecolor(color)
random.shuffle(colors)
for patch, color in zip(bplot2['boxes'], colors ):
    patch.set_facecolor(color)
random.shuffle(colors)
for patch, color in zip(bplot3['boxes'],  colors):
    patch.set_facecolor(color) 
random.shuffle(colors)
for patch, color in zip(bplot4['boxes'], colors):
    patch.set_facecolor(color)
#adding horizontal grid lines
ax[0,0].yaxis.grid(True)
ax[0,1].yaxis.grid(True)
ax[1,0].yaxis.grid(True)
ax[1,1].yaxis.grid(True)
plt.yticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show();


In [9]:
def embedding(data_column):
    """Creates embedding for the categorical data, i.e assign numerical value to unique values."""
    Unique_element = data_column.unique()
    mapping_elemnt = [int(x) for x in range(len(Unique_element)+1)]
    res = {Unique_element[i]: mapping_elemnt[i] for i in range(len(Unique_element))}
    return res

In [10]:
list1 = []
for i in rem_cols:
    temp = embedding(Processed_data[i])
    list1.append(temp)
    Processed_data[i] = Processed_data[i].map(temp)
#     print(Processed_data[i].unique())

In [11]:
Processed_data.isnull().sum()
Processed_data

Nationality_partner_man              0
Nationality_partner_woman            0
Age_partner_man                      0
Age_partner_woman                    0
Marriage_duration                    0
Place_of_birth_partner_man           0
Place_of_birth_partner_woman         0
Place_of_residence_partner_man       0
Place_of_residence_partner_woman     0
Monthly_income_partner_man_peso      0
Monthly_income_partner_woman_peso    0
Occupation_partner_man               0
Occupation_partner_woman             0
Employment_man                       0
Employment_woman                     0
Education_man                        0
Education_woman                      0
Num_Children                         0
Custody                              0
dtype: int64

,Nationality_partner_man,Nationality_partner_woman,Age_partner_man,Age_partner_woman,Marriage_duration,Place_of_birth_partner_man,Place_of_birth_partner_woman,Place_of_residence_partner_man,Place_of_residence_partner_woman,Monthly_income_partner_man_peso,Monthly_income_partner_woman_peso,Occupation_partner_man,Occupation_partner_woman,Employment_man,Employment_woman,Education_man,Education_woman,Num_Children,Custody
0,0,0,30.0,22.0,5.0,0,0,0,0,2000.0,1800.0,0,0,0,0,0,0,1,0
4,0,0,30.0,27.0,7.0,0,1,1,1,18000.0,3.5,1,1,1,1,1,1,2,1
5,0,0,32.0,29.0,11.0,1,2,0,0,3.0,3.5,2,0,1,1,0,1,2,1
6,0,0,36.0,35.0,15.0,0,1,0,2,3.0,3.5,2,2,1,1,1,1,2,1
7,0,0,31.0,32.0,6.0,2,3,0,0,3.0,3.5,3,3,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4867,0,0,35.0,34.0,3.0,464,1,157,0,12500.0,6000.0,71,0,1,0,1,0,0,0
4868,0,0,30.0,32.0,1.0,5,22,0,0,3.0,3.5,2,0,1,0,1,0,0,0
4869,0,0,59.0,56.0,35.0,0,1,26,0,20000.0,29000.0,1,32,1,0,1,0,2,0
4870,0,0,54.0,48.0,33.0,45,22,0,0,3.0,3.5,2,1,1,1,2,1,2,0


In [12]:
correlation = Processed_data.corr()
correlation
normalized_data = Processed_data[num_cols]

,Nationality_partner_man,Nationality_partner_woman,Age_partner_man,Age_partner_woman,Marriage_duration,Place_of_birth_partner_man,Place_of_birth_partner_woman,Place_of_residence_partner_man,Place_of_residence_partner_woman,Monthly_income_partner_man_peso,Monthly_income_partner_woman_peso,Occupation_partner_man,Occupation_partner_woman,Employment_man,Employment_woman,Education_man,Education_woman,Num_Children,Custody
Nationality_partner_man,1.000000,0.001160,0.004539,0.006957,-0.032641,0.176201,-0.001264,0.022345,-0.016960,0.000935,0.008332,0.018921,0.023375,-0.004697,-0.000833,0.013261,0.004366,-0.028534,-0.021783
Nationality_partner_woman,0.001160,1.000000,0.000252,-0.007718,-0.023964,-0.015208,0.168976,0.019726,0.012464,0.004121,0.015347,0.016691,0.016902,0.009851,-0.012214,-0.011907,-0.017248,-0.015735,-0.004298
Age_partner_man,0.004539,0.000252,1.000000,0.864083,0.717205,0.140299,0.104434,0.079557,0.035674,0.027952,0.047049,0.082616,0.044335,0.069158,0.083449,0.086556,0.086170,0.361972,-0.090875
Age_partner_woman,0.006957,-0.007718,0.864083,1.000000,0.757700,0.112810,0.090665,0.074147,0.027499,0.017722,0.065804,0.068641,0.054425,0.044052,0.067197,0.083337,0.072720,0.380689,-0.081658
Marriage_duration,-0.032641,-0.023964,0.717205,0.757700,1.000000,0.068606,0.053555,0.054778,0.025605,0.023544,0.027921,0.044158,-0.000410,0.052615,0.095681,0.068005,0.095693,0.550698,0.024100
Place_of_birth_partner_man,0.176201,-0.015208,0.140299,0.112810,0.068606,1.000000,0.148311,0.205085,0.048630,-0.009005,-0.029864,0.032151,-0.011374,0.038532,0.058764,0.083243,0.063517,0.049568,-0.016433
Place_of_birth_partner_woman,-0.001264,0.168976,0.104434,0.090665,0.053555,0.148311,1.000000,0.068550,0.130770,-0.017910,-0.028475,0.013703,-0.030813,0.025960,0.052794,0.072555,0.047843,0.057267,0.001764
Place_of_residence_partner_man,0.022345,0.019726,0.079557,0.074147,0.054778,0.205085,0.068550,1.000000,0.186368,-0.001906,-0.006831,0.053997,0.022513,0.053406,0.016499,0.020486,0.014084,0.027551,-0.006461
Place_of_residence_partner_woman,-0.016960,0.012464,0.035674,0.027499,0.025605,0.048630,0.130770,0.186368,1.000000,-0.005248,0.022633,0.001530,0.009722,-0.014649,-0.004736,-0.004630,-0.003533,0.042350,0.030370
Monthly_income_partner_man_peso,0.000935,0.004121,0.027952,0.017722,0.023544,-0.009005,-0.017910,-0.001906,-0.005248,1.000000,0.227611,0.007011,0.014176,-0.007350,-0.009244,-0.009962,-0.018145,0.010589,0.026182


In [13]:
#embedding the divorce_type of the dataset
Processed_data_array = np.array(Processed_data)
divorce_type = data['Type_of_divorce'] 
divorce_type_embedding = embedding(divorce_type)
divorce_type = divorce_type.map(divorce_type_embedding)
maritial_status = data.maritial_status

#cretating preety table.
x = PrettyTable()
x.field_names = ["Feature Name", "Score"]


# finding the important features of the dataset.
model = DecisionTreeRegressor()
# fit the model
model.fit(Processed_data_array, divorce_type)
# get importance
importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    x.add_row([cols[i],v])
x.align = "l"
print(x.get_string(sortby="Score"))

DecisionTreeRegressor()

+-----------------------------------+----------------------+
| Feature Name                      | Score                |
+-----------------------------------+----------------------+
| Nationality_partner_woman         | 0.0                  |
| Nationality_partner_man           | 0.001288719925366224 |
| Custody                           | 0.01581351712137498  |
| Education_woman                   | 0.01599622824927132  |
| Place_of_residence_partner_woman  | 0.018693988926225702 |
| Place_of_residence_partner_man    | 0.022871609475119948 |
| Education_man                     | 0.023148427486639565 |
| Employment_woman                  | 0.025848013904730432 |
| Employment_man                    | 0.04676612468399754  |
| Place_of_birth_partner_woman      | 0.051022978045361385 |
| Occupation_partner_man            | 0.05282164481871138  |
| Occupation_partner_woman          | 0.05486221708047829  |
| Place_of_birth_partner_man        | 0.062226059536866114 |
| Monthly_income_partner

#### Above score defines the major contributing factors: Num_Children (i.e no of childrens of a couple), Monthly income of man, Marraige duration, Age of both parteners.

In [14]:
#normalized of the processed dataset.
normalized_data = preprocessing.normalize(np.array(normalized_data))

#dividing the dataset into test and train.
trainX, testX, trainy, testy = train_test_split(normalized_data, maritial_status, test_size=0.1, random_state=42)

In [15]:
#aplying the classifiers for outliers of the processed dataset.
model = OneClassSVM(gamma='scale', nu=0.06)
model.fit(trainX)

OneClassSVM(nu=0.06)

In [16]:
# detect outliers in the test set
yhat = model.predict(testX)
# mark inliers 1, outliers -1
testy[testy == 1] = -1
testy[testy == 0] = 1
# calculate score
score = f1_score(testy, yhat, pos_label=-1)
print('F1 Score: %f' % score)

F1 Score: 0.000000


In [17]:
#divide the non-normalized dataset.
trainX, testX, trainy, testy = train_test_split(np.array(Processed_data[num_cols]), 
                                                divorce_type, test_size=0.6, random_state=42)

In [18]:
# define the decesion tree classifiers.
model = DecisionTreeClassifier()
clf = model.fit(trainX, trainy)
print(model.score(testX,  testy))

0.6345318056104307


In [19]:
def input_process(data):
    """Process input of the data"""
    temp_data = preprocessing.normalize(np.array([data]))
    temp = model.predict(temp_data)
    if temp==1:
        x = clf.predict([data])
        if x == 0:
            print("Possibility of Divorce of type Necesario.")
        else:
             print("Possibility of Divorce of type Voluntario.")
    else:
        print("No possibility of Divorce.")

Test for the input  value: 
Enter the following format: ['Age_partner_man', 'Age_partner_woman', 'Marriage_duration',
       'Monthly_income_partner_man_peso', 'Monthly_income_partner_woman_peso',
       'Num_Children']

In [20]:
#Taking an example:
X  = [float(x) for x in input("Enter Comma Seperated  6 values in the order define above :").split(",")]
input_process(X)

Enter Comma Seperated  6 values in the order define above :50,45,10,100000,10000,2
Possibility of Divorce of type Necesario.
